In [ ]:
 from google.colab import drive
 drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
tf = pd.read_pickle('gdrive/My Drive/data/shorelines/shoreline-positions.pkl')
metadata = pd.read_pickle('gdrive/My Drive/data/shorelines/sites-compressed.pkl')
outliers = pd.read_pickle('gdrive/My Drive/data/shorelines/outliers.pkl')

In [ ]:
metadata = metadata.loc[metadata['flag_sandy']==True]
metadata = metadata.loc[metadata['no_sedcomp']!=0]
metadata = metadata.loc[metadata['low_detect_shlines']!=0]
metadata = metadata.loc[metadata['err_changerate']!=0]
metadata = metadata.loc[metadata['err_timespan']!=0]

AttributeError: ignored

### Drop rows without lon/lat 

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

def df2gdf(df):
  crs = {"init": "epsg:4326"}
  return gpd.GeoDataFrame(df, crs=crs, geometry=df['geometry'])

print(f"Transects in metadata: {len(metadata))}")
gdf = df2gdf(metadata)
keep_indices = gdf.loc[gdf['geometry'].is_empty == False]['transect_id'].to_list()
metadata = metadata.loc[metadata["transect_id"].isin(keep_indices)]
print(f"Transects after dropping empty lon/lat: {len(metadata))}")

### Drop redundant data 
In the metadatset for example the time series; we hold that data in a separate frame. 

In [ ]:
metadata = metadata.drop(columns=['outliers_1', 'outliers_2', 'dt', 'dist'])

In [ ]:
tf = tf.loc[tf['transect_id'].isin(metadata['transect_id'].unique())]
print(f"Transects included in time series selection: {len(tf['transect_id'].unique())}")
print(f"Total rows in time series frame: {len(tf)}")

### Drop outliers

In [ ]:
outliers_1 = outliers.set_index('transect_id')['outliers_1'].to_dict()
outliers_2 = outliers.set_index('transect_id')['outliers_2'].to_dict()

In [ ]:
def chunck_it(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    return out

transects = list(tf['transect_id'].unique())
transect_chunks = chunck_it(transects, 100)

In [ ]:
import numpy as np

from datetime import timedelta, datetime
from tqdm.auto import tqdm

def drop_indices(series, outliers):
    #   idx = outliers[series.name]
    mask = np.ones(len(series), dtype=bool)
    mask[outliers] = False
    return series[mask]

tqdm.pandas()

tf_list = []
count = 0
for chunk in transect_chunks:
    count+=1
    print(f"Chunk: {count}/{len(transect_chunks)}..")
    
    tf_ = tf.loc[tf['transect_id'].isin(chunk)]
    tf_ = tf_.groupby('transect_id').progress_apply(lambda x: drop_indices(x, outliers_1[x.name]))
    tf_ = tf_.droplevel('transect_id')
    tf_ = tf_.groupby('transect_id').progress_apply(lambda x: drop_indices(x, outliers_2[x.name]))
    tf_ = tf_.droplevel('transect_id')
    tf_list.append(tf_) 

tf_clean = pd.concat(tf_list)

Chunk: 1/101..



Chunk: 2/101..


Exception ignored in: <bound method tqdm.__del__ of 100%|<bar/>| 6356/6356 [24:22<00:00,  4.34it/s]>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tqdm/std.py", line 1061, in __del__
    def __del__(self):
KeyboardInterrupt



Chunk: 3/101..



Chunk: 4/101..


...


Chunk: 97/101..



Chunk: 98/101..



Chunk: 99/101..



Chunk: 100/101..



Chunk: 101/101..


In [ ]:
print(tf_clean.shape)
tf_clean.head()


# tf_clean.to_csv('gdrive/My Drive/data/shorelines/time-series-selection.csv', index=False, header=True)
# tf_clean.to_pickle("gdrive/My Drive/data/shorelines/time-series-selection.pkl")

(16140492, 3)


,dt,dist,transect_id
415526,14.001657,784.996887,BOX_028_002_1
415527,16.000328,737.042358,BOX_028_002_1
415531,20.000410,740.190002,BOX_028_002_1
415532,21.002485,787.516907,BOX_028_002_1
415533,22.001822,806.178711,BOX_028_002_1
